# Feature Engineering with Snowflake

This notebook focuses on creating the features required for the churn prediction model. The feature engineering logic is primarily executed using SQL within Snowflake to leverage its powerful analytical capabilities.

## Objectives
1. Connect to Snowflake
2. Create the `CUSTOMER_360` analytical view
3. Create the `CUSTOMER_FEATURES` table for machine learning
4. Verify the created tables and views

In [ ]:
# Import libraries
import sys
sys.path.append("/home/ubuntu/snowflake_ds_project")

import pandas as pd

from src.snowflake_connector import SnowflakeConnector
from src.data_loader import DataLoader
from config import config

print("Libraries imported successfully!")

## 1. Connect to Snowflake

In [ ]:
# Validate configuration
config.validate()

# Create connector and connect
connector = SnowflakeConnector(config.get_connection_params())
connector.connect()

print("Connected to Snowflake successfully!")

## 2. Create `CUSTOMER_360` View

This view consolidates data from the `CUSTOMERS`, `TRANSACTIONS`, and `CUSTOMER_ACTIVITY` tables to create a comprehensive, 360-degree view of each customer.

In [ ]:
customer_360_query = """
CREATE OR REPLACE VIEW ECOMMERCE_DB.ANALYTICS.CUSTOMER_360 AS
SELECT 
    c.customer_id,
    c.registration_date,
    c.age,
    c.gender,
    c.country,
    c.membership_tier,
    ca.last_login_date,
    ca.page_views,
    ca.support_tickets,
    ca.email_opened,
    COUNT(DISTINCT t.transaction_id) as total_transactions,
    SUM(t.amount) as total_spend,
    AVG(t.amount) as avg_transaction_value,
    MAX(t.transaction_date) as last_transaction_date,
    DATEDIFF(day, MAX(t.transaction_date), CURRENT_DATE()) as days_since_last_purchase
FROM ECOMMERCE_DB.RAW_DATA.CUSTOMERS c
LEFT JOIN ECOMMERCE_DB.RAW_DATA.CUSTOMER_ACTIVITY ca 
    ON c.customer_id = ca.customer_id
LEFT JOIN ECOMMERCE_DB.RAW_DATA.TRANSACTIONS t 
    ON c.customer_id = t.customer_id
GROUP BY 
    c.customer_id, c.registration_date, c.age, c.gender, 
    c.country, c.membership_tier, ca.last_login_date, 
    ca.page_views, ca.support_tickets, ca.email_opened;
"""

try:
    print("Creating CUSTOMER_360 view...")
    connector.execute_query(customer_360_query)
    print("CUSTOMER_360 view created successfully.")
except Exception as e:
    print(f"Error creating view: {e}")

In [ ]:
# Preview the view
preview_query = "SELECT * FROM ECOMMERCE_DB.ANALYTICS.CUSTOMER_360 LIMIT 10;"
customer_360_df = connector.execute_query_to_df(preview_query)
customer_360_df

## 3. Create `CUSTOMER_FEATURES` Table

This table transforms the data from the `CUSTOMER_360` view into a format suitable for machine learning. This includes:
- Encoding categorical variables (e.g., `gender`, `membership_tier`)
- Calculating time-based features (e.g., `days_since_registration`)
- Handling null values
- Defining the target variable (`is_churned`)

In [ ]:
# Use the DataLoader to create the feature table
loader = DataLoader(connector)

try:
    loader.create_feature_table()
except Exception as e:
    print(f"Error creating feature table: {e}")

In [ ]:
# Preview the feature table
preview_query = "SELECT * FROM ECOMMERCE_DB.FEATURES.CUSTOMER_FEATURES LIMIT 10;"
features_df = connector.execute_query_to_df(preview_query)
features_df

## 4. Verify Creations

In [ ]:
# Verify row counts
customer_count = connector.get_row_count("CUSTOMERS", schema="RAW_DATA")
feature_count = connector.get_row_count("CUSTOMER_FEATURES", schema="FEATURES")

print(f"Total customers in RAW_DATA.CUSTOMERS: {customer_count:,}")
print(f"Total customers in FEATURES.CUSTOMER_FEATURES: {feature_count:,}")
print(f"Row counts match: {customer_count == feature_count}")

In [ ]:
# Check churn distribution in the feature table
churn_dist_query = """
SELECT 
    is_churned,
    COUNT(*) as customer_count,
    (COUNT(*) / (SELECT COUNT(*) FROM ECOMMERCE_DB.FEATURES.CUSTOMER_FEATURES)) * 100 as percentage
FROM ECOMMERCE_DB.FEATURES.CUSTOMER_FEATURES
GROUP BY is_churned;
"""

churn_df = connector.execute_query_to_df(churn_dist_query)
churn_df

## 5. Cleanup

In [ ]:
# Disconnect from Snowflake
connector.disconnect()
print("Disconnected from Snowflake")